In [128]:

from nltk.corpus import reuters
import re
import spacy


## Data scrapping for oil products and Oil companies

In [368]:
import requests
from bs4 import BeautifulSoup
import re
import spacy
import csv
import json


In [106]:
'''def get_ent(stringg):
    doc = nlp(stringg)
    for ent in doc.ents:
        if ent.label_ is 'LOC':
            print([ent.text,ent.label_])
        else:
            print(ent.text)'''

In [389]:
def scrape_prodructs():

    #scrapping the oil-based products
    _products = []

    r= requests.get('https://innovativewealth.com/inflation-monitor/what-products-made-from-petroleum-outside-of-gasoline/')
    soup = BeautifulSoup(r.text,'html.parser')
    for i in soup.find_all('td'):
        _products.append(i.string)

        
    #getting the none type out of the list
    _products = filter(None, _products)

    #converting the elements in list to lowercase
    _products = [p.lower() for p in _products]
    print(_products)

    #saving the data as csv format
    with open('scrapped_oil_products.csv','w') as f:
        writer = csv.writer(f)
        writer.writerow(_products)
    
    return _products

_products = scrape_prodructs()

['solvents', 'diesel fuel', 'motor oil', 'bearing grease', 'ink', 'floor wax', 'ballpoint pens', 'football cleats', 'upholstery', 'boats', 'insecticides', 'bicycle tires', 'sports car bodies', 'nail polish', 'fishing lures', 'dresses', 'tires', 'golf bags', 'perfumes', 'cassettes', 'dishwasher parts', 'tool boxes', 'shoe polish', 'motorcycle helmet', 'caulking', 'petroleum jelly', 'transparent tape', 'faucet washers', 'antiseptics', 'clothesline', 'curtains', 'food preservatives', 'basketballs', 'vitamin capsules', 'antihistamines', 'purses', 'shoes', 'dashboards', 'cortisone', 'deodorant', 'footballs', 'putty', 'dyes', 'panty hose', 'refrigerant', 'percolators', 'life jackets', 'rubbing alcohol', 'linings', 'skis', 'tv cabinets', 'shag rugs', 'electrician’s tape', 'tool racks', 'car battery cases', 'epoxy', 'paint', 'mops', 'slacks', 'insect repellent', 'oil filters', 'umbrellas', 'yarn', 'fertilizers', 'hair coloring', 'roofing', 'toilet seats', 'fishing rods', 'lipstick', 'denture a

In [384]:
def scrape_oil_companies():
    #scrapping 220 oil companies
    _com = []
    #page 1 and 3 
    for i in range(1,4):
        url = f'https://companiesmarketcap.com/oil-gas/largest-oil-and-gas-companies-by-market-cap/?page={i}'
        print(url)
        r= requests.get(url)
        soup = BeautifulSoup(r.text,'html.parser')
        for j in soup.find_all('div',{'class':'company-name'}):
            _com.append(j.string)

    #lowercase the company names
    _com = [c.lower() for c in _com]
    _com = [c.replace('\n','') for c in _com]

    #save the data as csv format
    with open('scrapped_oil_companies.csv','w',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(_com)

    return _com


In [136]:
#check if doc has the 200 companies that were scrapped
def has_company_name(doc):
    for token in doc:
        if token.lower_ in _com:
            if token.pos_ != 'VERB':
                return True
    return False

## Importing the crude dataset 

In [381]:
#cleaned text using regex, splitting every sentence into elements of a list
def clean(text):
    text = text.replace('&lt;','<') # html escape
    text = re.sub(r'[<>]', '"', text) # quotation marks instead of <>
    text = re.sub(r'[ ]*"[A-Z\.]+"', '', text) # drop stock symbols
    text = re.sub(r'[ ]*\([A-Z\.]+\)', '', text) # drop stock symbols
    text = re.sub(r'\bdlr(s?)\b', r'dollar\1', text, flags=re.I)
    text = re.sub(r'\bmln(s?)\b', r'million\1', text, flags=re.I)
    text = re.sub(r'\bpct\b', r'%', text, flags=re.I)
    text = re.sub(r'"', r'', text) # quotation marks
    text = re.sub(r'\s+', ' ', text) # multiple whitespace by one
    text = text.lower()
    arr = []
    arr = text.split('. ')
    return arr
    
    
#load nlp object
nlp = spacy.load('en_core_web_lg') #, disable=['parser', 'tagger'])

#load reuters crude id
reuters_fileids_crudes = reuters.fileids(categories=['crude'])

#load raw content of the dataset
raww = [reuters.raw(i) for i in reuters_fileids_crudes]

#splitting everything in sentences(split if there is a . and space)-> ex: 'the end. ' and cleaning the raw text.  
per_sentence_text = []
for line in raww:
    per_sentence_text.append(clean(line))


## Get training data

In [438]:

    
#parsing the text into a training format
def parse_train_data_products(txt,_products=_products):
    """
    loops through the product list. if it finds a product in the list that's also in the text, then it'll look for the first char position and the last one. 
    Then it'll asign a PRODUCT entitie to it.
    result and structure example -> ('gasoline supplies would also be limited', {'entities': [(0, 8, 'PRODUCT')]})
    at char index 0 until 8 we have the word 'gasoline' wich is in the product list.
    """
    for pro in _products:
        if pro in txt:
            start_char = txt.find(pro)
            end_char = txt.find(pro)+len(pro)
            productt = [(start_char,end_char,'PRODUCT')]
            return(txt,{'entities':productt})

In [393]:
#check if doc has product names that were scraped
def has_product_name(text, _products=_products,nlp=nlp):
    """this function checks of the words are in the product-list.
    """
    doc = nlp(text)
    for token in doc:
        #checks of word in product list
        if token.lower_ in _products:
            if token.pos_ != 'VERB':
                return True
    return False   

In [439]:
#getting the training data
_training_data = []    

#looping through all the paragraphs
for i in range(len(per_sentence_text)):
    #looping through all the sentences
    for j in range(len(per_sentence_text[i])):
        #checking if sentences have the product name in them
        if has_product_name(per_sentence_text[i][j]):
            txt = str(per_sentence_text[i][j])
            #if they have the product name, we take the sentence and append it to the training data
            _training_data.append(parse_train_data_products(txt))


In [442]:
#saving training data into a json file
with open('training_data.json','w') as f:
    json.dump(_training_data,f)

In [437]:
#just a test script!

txt = 'iraqi naval units and forces defending an offshore terminal sank six iranian out of 28 iranian boats attempting to attack an offshore terminal, the communique said'

for pro in _products:
    if pro in txt:
        start_char = txt.find(pro)
        print(start_char)
        end_char = txt.find(pro)+len(pro)
        productt = [(start_char,end_char,'PRODUCT')]
        print (txt,{'entities':productt})

95
iraqi naval units and forces defending an offshore terminal sank six iranian out of 28 iranian boats attempting to attack an offshore terminal, the communique said {'entities': [(95, 100, 'PRODUCT')]}


## TRAINING LOOP


In [338]:
#creating a blank 
def create_blank_nlp(trainig_data):
    """
    create new pipeline blank pipeline. To my understanding this step is necessary because this way the model will be trained faster
    """

    #create blank nlp
    nlp = spacy.blank('en')
    #create blank ner pipe
    ner = nlp.create_pipe('ner')
    #attach ner pipeline to nlp opbject as the last pipe
    nlp.add_pipe('ner',last=True)
    
    ner = nlp.get_pipe('ner')
    #loop through the training data
    for _, annotations in trainig_data:

        #get the entities and add the label name to the created pipeline
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])#ent[2] -> 'PRODUCT'
    
    return nlp

In [345]:
import random
from spacy.training.example import Example

_nlp = create_blank_nlp(_training_data)
optimizer = _nlp.begin_training()
for i in range(10):
    random.shuffle(_training_data)
    losses ={}
    for text, annotations in _training_data:
        ex = Example.from_dict(nlp.make_doc(text),annotations)
        _nlp.update([ex],sgd=optimizer, losses=losses)
    print(f'iteration {i} - {losses}')

iteration 0 - {'ner': 257.7574053941867}
iteration 1 - {'ner': 23.359386698848684}
iteration 2 - {'ner': 24.805739280650158}
iteration 3 - {'ner': 19.909211754824483}
iteration 4 - {'ner': 17.051065559113887}
iteration 5 - {'ner': 17.457896092222207}
iteration 6 - {'ner': 14.93686104913363}
iteration 7 - {'ner': 14.066386426224916}
iteration 8 - {'ner': 13.402001827184426}
iteration 9 - {'ner': 15.746401900296117}


In [341]:
_training_data[0]

('gasoline supplies would also be limited', {'entities': [(0, 8, 'PRODUCT')]})

In [347]:


import spacy
from spacy import displacy

doc = _nlp('gasoline supplies would also be limited')
displacy.serve(doc, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
